In [18]:
%load_ext autoreload
%reload_ext autoreload
%autoreload 2

target_url = "https://bsr.sudrf.ru/bigs/portal.html#%7B%22type%22:%22MULTIQUERY%22,%22multiqueryRequest%22:%7B%22queryRequests%22:%5B%7B%22type%22:%22Q%22,%22request%22:%22%7B%5C%22mode%5C%22:%5C%22EXTENDED%5C%22,%5C%22typeRequests%5C%22:%5B%7B%5C%22fieldRequests%5C%22:%5B%7B%5C%22name%5C%22:%5C%22case_document_result_date%5C%22,%5C%22operator%5C%22:%5C%22B%5C%22,%5C%22query%5C%22:%5C%222018-01-01T00:00:00%5C%22,%5C%22sQuery%5C%22:%5C%222018-12-31T00:00:00%5C%22%7D,%7B%5C%22name%5C%22:%5C%22case_document_category_article_cat%5C%22,%5C%22operator%5C%22:%5C%22SEW%5C%22,%5C%22query%5C%22:%5C%22%D0%A1%D1%82%D0%B0%D1%82%D1%8C%D1%8F%20229.1%20%D0%A7%D0%B0%D1%81%D1%82%D1%8C%203%5C%22%7D%5D,%5C%22mode%5C%22:%5C%22AND%5C%22,%5C%22name%5C%22:%5C%22common%5C%22,%5C%22typesMode%5C%22:%5C%22AND%5C%22%7D%5D%7D%22,%22operator%22:%22AND%22,%22queryRequestRole%22:%22CATEGORIES%22%7D%5D%7D,%22sorts%22:%5B%7B%22field%22:%22score%22,%22order%22:%22desc%22%7D%5D,%22simpleSearchFieldsBundle%22:%22default%22,%22filterGroups%22:%5B%7B%22filterQueries%22:%5B%7B%22field%22:%22case_court_type%22,%22query%22:%22%D0%A0%D0%B0%D0%B9%D0%BE%D0%BD%D0%BD%D1%8B%D0%B9,%20%D0%B3%D0%BE%D1%80%D0%BE%D0%B4%D1%81%D0%BA%D0%BE%D0%B9,%20%D0%BC%D0%B5%D0%B6%D1%80%D0%B0%D0%B9%D0%BE%D0%BD%D0%BD%D1%8B%D0%B9%20%D1%81%D1%83%D0%B4%22,%22not%22:false%7D%5D,%22groupMode%22:%22OR%22,%22not%22:false%7D%5D%7D"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
from web_parser import Page

In [20]:
p = Page(target_url)

In [22]:
p.current_page_case_ids

['0d9bd36643ac3b90550ac542d06543dc',
 '4557c0fe018710615cd8f532b5d99e1d',
 '3b7c279490c063beca4fc585fe54ec07',
 'b1b641a77f1c480c8cb0e79990cf6b22',
 '105483a4ad9e7637977646308d8249c2',
 '035bbfff397da055f7662f4f9b5b4dfb',
 '814165e3bc803f8c81b95e12070f7d45',
 'dcf215c7ec2f3d0712623bacd0deb608',
 '11b96521738683e94f20224b10faafbc',
 'e367e6272547affd28d896e38495594a']

In [21]:
p.current_page_case_json_dicts

[{'document': {'boostRating': 0,
   'fields': [{'actualValue': None,
     'comment': 'Уровень суда',
     'dateValue': None,
     'doubleValue': None,
     'highlighting': [],
     'languages': None,
     'linkField': None,
     'linkName': None,
     'linkValue': None,
     'longValue': None,
     'markup': [],
     'name': 'case_court_type_cat',
     'presentationName': None,
     'tableCols': None,
     'value': 'Районный, городской, межрайонный суд',
     'valueHash': None,
     'valueWOHL': 'Районный, городской, межрайонный суд'},
    {'actualValue': None,
     'comment': 'Дела с текстом судебного акта',
     'dateValue': None,
     'doubleValue': None,
     'languages': None,
     'linkField': None,
     'linkName': None,
     'linkValue': None,
     'longValue': 1,
     'name': 'case_user_document_num',
     'presentationName': None,
     'tableCols': None,
     'value': None,
     'valueWOHL': '1'},
    {'actualValue': None,
     'comment': 'Округ РФ',
     'dateValue': None,
 

In [12]:
import requests

headers = {"Content-Type": "application/json"}
data = {"request": {"id": "0d9bd36643ac3b90550ac542d06543dc"}}
resp = requests.post("https://bsr.sudrf.ru/bigs/showDocument.action",
                     json=data,
                     headers=headers)

In [17]:
resp.json()

{'document': {'boostRating': 0,
  'fields': [{'actualValue': None,
    'comment': 'Уровень суда',
    'dateValue': None,
    'doubleValue': None,
    'highlighting': [],
    'languages': None,
    'linkField': None,
    'linkName': None,
    'linkValue': None,
    'longValue': None,
    'markup': [],
    'name': 'case_court_type_cat',
    'presentationName': None,
    'tableCols': None,
    'value': 'Районный, городской, межрайонный суд',
    'valueHash': None,
    'valueWOHL': 'Районный, городской, межрайонный суд'},
   {'actualValue': None,
    'comment': 'Дела с текстом судебного акта',
    'dateValue': None,
    'doubleValue': None,
    'languages': None,
    'linkField': None,
    'linkName': None,
    'linkValue': None,
    'longValue': 1,
    'name': 'case_user_document_num',
    'presentationName': None,
    'tableCols': None,
    'value': None,
    'valueWOHL': '1'},
   {'actualValue': None,
    'comment': 'Округ РФ',
    'dateValue': None,
    'doubleValue': None,
    'highli

In [59]:
case = json.loads(resp.text)
case

{'document': {'boostRating': 0,
  'fields': [{'actualValue': None,
    'comment': 'Уровень суда',
    'dateValue': None,
    'doubleValue': None,
    'highlighting': [],
    'languages': None,
    'linkField': None,
    'linkName': None,
    'linkValue': None,
    'longValue': None,
    'markup': [],
    'name': 'case_court_type_cat',
    'presentationName': None,
    'tableCols': None,
    'value': 'Районный, городской, межрайонный суд',
    'valueHash': None,
    'valueWOHL': 'Районный, городской, межрайонный суд'},
   {'actualValue': None,
    'comment': 'Дела с текстом судебного акта',
    'dateValue': None,
    'doubleValue': None,
    'languages': None,
    'linkField': None,
    'linkName': None,
    'linkValue': None,
    'longValue': 1,
    'name': 'case_user_document_num',
    'presentationName': None,
    'tableCols': None,
    'value': None,
    'valueWOHL': '1'},
   {'actualValue': None,
    'comment': 'Округ РФ',
    'dateValue': None,
    'doubleValue': None,
    'highli